# Reading outflow Fraser file

In [1]:
%matplotlib inline

import sys
import xarray as xr
import numpy as np
import os
import math
from datetime import datetime, timedelta
from parcels import FieldSet, Field, VectorField, ParticleSet, JITParticle, ErrorCode, ParcelsRandom

sys.path.append('/home/jvalenti/MOAD/analysis-jose/Source')
from Kernels_fibers import DeleteParticle, AdvectionRK4_3D, Stokes_drift, Beaching, Unbeaching,Buoyancy,turb_mix
from OP_functions_fibers import *

In [2]:
# Define paths
local = 0 #Set to 0 when working on server
paths = path(local)

In [3]:
path_NEMO = make_prefix(datetime(2018, 1, 10), paths['NEMO'])
coord=xr.open_dataset(paths['coords'],decode_times=False)
WW3 = xr.open_dataset(get_WW3_path(datetime(2018, 1, 10)))
batt=xr.open_dataset(paths['mask'],decode_times=False)
Fraser=xr.open_dataset(get_Fraser_path(datetime(2020, 8, 1)))

In [96]:
lat,lon=[49.219912, -122.873304]
jjii = xr.open_dataset('~/MOAD/grid/grid/grid_from_lat_lon_mask999.nc')
jj = jjii.jj.sel(lats=lat, lons=lon, method='nearest').item()
ii = jjii.ii.sel(lats=lat, lons=lon, method='nearest').item()

## Define and save mask for distance to coast

In [5]:
Dat=xr.open_dataset(paths['coords'],decode_times=False)
outf_lat=Dat['nav_lat'][445,304]
outf_lon=Dat['nav_lon'][445,304]
clon, clat = [float(outf_lon)],[float(outf_lat)]  # choose horizontal centre of the particle cloud

#fras_lat=Dat['nav_lat'][500,394]
#fras_lon=Dat['nav_lon'][500,394]
#flon, flat = [float(outf_lon)],[float(outf_lat)] 
flat,flon = [49.198421], [-123.131407]

## Definitions

In [6]:
start = datetime(2020, 8, 1) #Start date
# Set Time length [days] and timestep [seconds]
length = 5
duration = timedelta(days=length)
dt = 90 #toggle between - or + to pick backwards or forwards
N = 1 # number of deploying locations
n = 1 # 1000   # number of particles per location
dmin = 4 #minimum depth
dd = 2 #max depth difference from dmin
x_offset, y_offset, zvals = p_deploy(N,n,dmin,dd)

In [7]:
from parcels import Variable

class MPParticle(JITParticle):        
    ro = Variable('ro', initial = 1025)           
    diameter = Variable('diameter', initial = 2.16e-5)
    Sdd = Variable('SDD', initial = 3.34e-6)
    Sdl = Variable('SDL', initial = 24.4e-5)
    length = Variable('length', initial = 45e-5)
    Lb = Variable('Lb', initial = 0.23)  #days needed in days for particle to have 67% probability of beaching if in beaching zone (500m)
    Db = Variable('Db', initial = 33) #Distance at which particles can randomly beach.
    Ub = Variable('Ub', initial = 69)  #days to have 67% probability of unbeaching
    beached = Variable('beached', initial = 0)
    #Ws = Variable('Ws', initial =  2.3e-3) #200m/dia
    tau = Variable('tau', initial =  0) # track age particle
    fratio= Variable('fratio', initial =  0.0)

In [8]:
lon = np.zeros([N,n])
lat = np.zeros([N,n])
for i in range(N):
    lon[i,:]=(clon[i] + x_offset[i,:])
    lat[i,:]=(clat[i] + y_offset[i,:])
z = zvals

In [9]:
#Set start date time and the name of the output file
name = 'fibreS2' #name output file
daterange = [start+timedelta(days=i) for i in range(length)]
fn =  name + '_'.join(d.strftime('%Y%m%d')+'_1n' for d in [start, start+duration]) + '.nc'
outfile = os.path.join(paths['out'], fn)
print(outfile)

/home/jvalenti/MOAD/results/fibreS220200801_1n_20200806_1n.nc


## Simulation

In [10]:
#Fill in the list of variables that you want to use as fields
varlist=['U','V','W','R']
filenames,variables,dimensions=filename_set(start,length,varlist,local)
field_set=FieldSet.from_nemo(filenames, variables, dimensions, allow_time_extrapolation=True)
varlist=['US','VS','WL']
filenames,variables,dimensions=filename_set(start,length,varlist,local)

us = Field.from_netcdf(filenames['US'], variables['US'], dimensions,allow_time_extrapolation=True)
vs = Field.from_netcdf(filenames['VS'], variables['VS'], dimensions,allow_time_extrapolation=True)
wl = Field.from_netcdf(filenames['WL'], variables['WL'], dimensions,allow_time_extrapolation=True)
filenames['US']
field_set.add_field(us)
field_set.add_field(vs)
field_set.add_field(wl)
field_set.add_vector_field(VectorField("stokes", us, vs, wl))

filenames,variables,dimensions=filename_set(start,length,['Bathy'],local)
Bth = Field.from_netcdf(filenames['Bathy'], variables['Bathy'], dimensions,allow_time_extrapolation=True)
field_set.add_field(Bth)

filenames,variables,dimensions=filename_set(start,length,['FS'],local)
Fraser = Field.from_netcdf(filenames['FS'], variables['FS'], dimensions,allow_time_extrapolation=True,timestamps=get_timestamps(start,length))
field_set.add_field(Fraser)

filenames,variables,dimensions=filename_set(start,length,['Kz'],local)
Kz = Field.from_netcdf(filenames['Kz'], variables['Kz'], dimensions,allow_time_extrapolation=True)
field_set.add_field(Kz)

         It will be opened with no decoding. Filling values might be wrongly parsed.


In [11]:
# #Load Salish output as fields
#field_set = FieldSet.from_nemo(filenames, variables, dimensions, allow_time_extrapolation=True)
pset = ParticleSet.from_list(field_set, MPParticle, lon=lon, lat=lat, depth=z, repeatdt = timedelta(hours=1))

k_sink = pset.Kernel(Buoyancy)
k_waves = pset.Kernel(Stokes_drift)
k_beach = pset.Kernel(Beaching)
k_unbeach = pset.Kernel(Unbeaching)
k_turb = pset.Kernel(turb_mix)


pset.execute(AdvectionRK4_3D + k_sink + k_waves + k_beach + k_unbeach+ k_turb,
             runtime=duration, 
             dt=dt,
             output_file=pset.ParticleFile(name=outfile, outputdt=timedelta(hours=1)),
             recovery={ErrorCode.ErrorOutOfBounds: DeleteParticle})

INFO: Compiled ArrayMPParticleAdvectionRK4_3DBuoyancyStokes_driftBeachingUnbeachingturb_mix ==> /tmp/parcels-2894/lib48c260a05007a7cfcab4467337dbb707_0.so


-0.000000
-0.000000
-0.000000
0.000000
0.000000
0.000000
0.000000
0.000000
0.000000
0.000000
0.000000
0.000000
0.000000
0.000000
0.000000
0.000000
0.000000
0.000000
0.000000
0.000000
0.000000
0.000000
0.000000
0.000000
0.000000
0.000000
0.000000
0.000000
0.000000
0.000000
0.000000
0.000000
0.000000
0.000000
0.000000
0.000000
0.000000
0.000000
0.000000
0.000000
0.000000
0.000000
0.000000
0.000000
0.000000
0.000000
0.000000
0.000000
0.000000
0.000000
0.000000
0.000000
0.000000
0.000000
0.000000
0.000000
0.000000
0.000000
0.000000
0.000000
0.000000
0.000000
0.000000
0.000000
0.000000
0.000000
0.000000
0.000000
0.000000
0.000000
0.000000
0.000000
0.000000
0.000000
0.000000
0.000000
0.000000
0.000000
0.000000
0.000000
0.000000
0.000000
0.000000
0.000000
0.000000
0.000000
0.000000
0.000000
0.000000
0.000000
0.000000
0.000000
0.000000
0.000000
0.000000
0.000000
0.000000
0.000000
0.000000
0.000000
0.000000
0.000000
0.000000
0.000000
0.000000
0.000000
0.000000
0.000000
0.000000
0.000000
0.00000

KeyboardInterrupt: 